In [ ]:
#Load libs

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000 
from PIL import Image, ImageOps
import fcsparser

#set up working dir

directory = str("/mnt/SERVER-CRCT-STORAGE/CRCT_Imagin/CORDELIER Pierre/HaloData/21-003.IMMCORE.C1v1/Halo archive 2021-12-07 10-09 - v3.3.2541/ObjectData")
outpath = '/home/mouneem/tysserand/CORDELIER_PIERRE/'

In [ ]:
# FCS TO CSV
# iterate over files in
# that directory
i = 0
split_chars = ".czi"
for filename in os.listdir(directory)[:]:
    i += 1
    f = os.path.join(directory, filename)
    # checking if it is a file
    filename_csv = filename.split(split_chars)[0] + ".csv"
    if os.path.isfile(f) and 'fcs' in filename:

        meta, data = fcsparser.parse(f, reformat_meta=True)
        type(data)
        #df1 = df[['a', 'b']] // df1 = df.iloc[:, 0:2] 

        data["x"] = (data["XMax"] + data["XMin"] ) / 2
        data["y"] = (data["YMax"] + data["YMin"] ) / 2

        col_is_marker = []
        for col in list(data.columns):
            if col.count('+') + col.count('-') > 1:
                col_is_marker.append(True)
            else:
                col_is_marker.append(False)
       # print( data.columns  )


        marker = data.loc[:, col_is_marker]
        print(marker.columns)
        M1_markers = marker['CD206+CD163+CD68-CD16-'].astype(bool) | marker['CD206+CD163+CD68-CD16+'].astype(bool)
        M2a_markers = marker['CD206+CD163+CD68-CD16-'].astype(bool) | marker['CD206+CD163+CD68-CD16+'].astype(bool) 
        M2c_markers = marker['CD206-CD163+CD68-CD16+'].astype(bool) 

        classes = ['Other']*data.shape[0]

        for i in range(len(classes)):
            if M1_markers.values[i]:
                classes[i] = 'M1'
            elif M2c_markers.values[i]:
                classes[i] = 'M2c'
            elif M2a_markers.values[i]:
                classes[i] = 'M2a'

        from collections import Counter
        print(Counter(classes))

        data["class"] = classes

        nodes = data[['x','y',"class"]]
        nodes.to_csv(outpath + "./csv_M1_M2/"+filename_csv,  mode='a', header = False)


        # ratio = 10  CD206-CD163+CD68-CD16-
        # nodes = nodes.sample(n = round(len(nodes) / ratio) )
        # coords = nodes.loc[:,['x','y']].values

        # plt.scatter(coords[:,0] , coords[:,1] ,   zorder=10, s = 1)
    
        # plt.savefig(outpath+filename_csv+'.png')
        
        # plt.cla()
